In [1]:
%%init_spark
launcher.conf.spark.app.name = "avinash"
launcher.conf.spark.local.dir = "/data/tmp/spark"
launcher.conf.spark.driver.memory = "40g"
launcher.conf.spark.executor.memory = "50g"
launcher.conf.spark.executors = "21"
launcher.conf.spark.executor.cores = "21"
launcher.conf.spark.sql.shuffle.partitions = 210
launcher.conf.spark.sql.shuffle.minPartitions = 20
launcher.conf.spark.ui.showConsoleProgress = "true"
launcher.conf.spark.yarn.keytab = "/home/jioapp/aditya/jiomart_cluster/couture.keytab"
launcher.conf.spark.yarn.principal = "couture@COUTURE.AI"
launcher.conf.spark.driver.extraJavaOptions = "-Djava.security.krb5.conf=/home/jioapp/aditya/jiomart_cluster/krb5.conf"
launcher.conf.spark.executor.extraJavaOptions = "-Djava.security.krb5.conf=/home/jioapp/aditya/jiomart_cluster/krb5.conf"

In [2]:
import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._

Intitializing Scala interpreter ...

Spark Web UI available at http://10.166.181.219:4043
SparkContext available as 'sc' (version = 3.3.0, master = local[*], app id = local-1750414972267)
SparkSession available as 'spark'


import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._


In [17]:
def readP(path:String)={
    var df = spark.read.parquet(raw"hdfs://10.144.96.170:8020/$path")
    df
}

readP: (path: String)org.apache.spark.sql.DataFrame


In [18]:
var categoryAffinityDF=readP("/data1/archive/avinash/RECO/userFeaturesFinal")

categoryAffinityDF: org.apache.spark.sql.DataFrame = [user_id: string, avg_bag_value: double ... 18 more fields]


In [19]:
var explodeAffinity = categoryAffinityDF
.withColumn("exploded_affinity",explode(col("attribute_affinity_brandname")))
.withColumn("brandname",col("exploded_affinity.attribute"))
.withColumn("score",col("exploded_affinity.affinity_score"))

explodeAffinity: org.apache.spark.sql.DataFrame = [user_id: string, avg_bag_value: double ... 21 more fields]


In [50]:
var finalUserFeatures = explodeAffinity
.drop("l1codes_purchases","l2codes_purchases","bricks_purchased","attribute_affinity_brandname","exploded_affinity")

finalUserFeatures: org.apache.spark.sql.DataFrame = [user_id: string, avg_bag_value: double ... 16 more fields]


In [51]:
finalUserFeatures.printSchema

root
 |-- user_id: string (nullable = true)
 |-- avg_bag_value: double (nullable = true)
 |-- avg_cart_to_checkout_hours: double (nullable = true)
 |-- avg_cart_to_checkout_days: double (nullable = true)
 |-- avg_order_interval_secs: double (nullable = true)
 |-- avg_order_interval_days: double (nullable = true)
 |-- avg_discount: double (nullable = true)
 |-- lifetime_order_value: double (nullable = true)
 |-- active_weeks: long (nullable = true)
 |-- total_orders: long (nullable = true)
 |-- avg_orders_per_week: double (nullable = true)
 |-- num_l1codes_purchased: integer (nullable = true)
 |-- num_l2codes_purchased: integer (nullable = true)
 |-- num_bricks_purchased: integer (nullable = true)
 |-- total_no_of_returns: long (nullable = true)
 |-- total_value_of_return: double (nullable = true)
 |-- brandname: string (nullable = true)
 |-- score: double (nullable = true)



In [59]:
var df = finalUserFeatures.na.drop()

df: org.apache.spark.sql.DataFrame = [user_id: string, avg_bag_value: double ... 16 more fields]


In [53]:
finalUserFeatures
.count()

res14: Long = 93266106


In [54]:
finalUserFeatures
.na.drop()
.count()

res15: Long = 1480809


In [60]:
import org.apache.spark.sql.{DataFrame, SparkSession}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._

def normalize_column(df: DataFrame, column: String): DataFrame = {
  val stats = df.agg(min(col(column)).as("min"), max(col(column)).as("max")).first()
  val minVal = stats.getAs[Any]("min").toString.toDouble
  val maxVal = stats.getAs[Any]("max").toString.toDouble

  if (minVal == maxVal) {
    df.withColumn(column, lit(0.0))  // All values are same
  } else {
    df.withColumn(column, (col(column).cast("double") - minVal) / (maxVal - minVal))
  }
}

def normalize_all_columns(df: DataFrame, excludeCols: Seq[String] = Seq("user_id", "brandname", "score")): DataFrame = {
  val numericCols = df.schema.fields
    .filter(f => f.dataType.isInstanceOf[NumericType] && !excludeCols.contains(f.name))
    .map(_.name)

  numericCols.foldLeft(df)((accDF, colName) => normalize_column(accDF, colName))
}


import org.apache.spark.sql.{DataFrame, SparkSession}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
normalize_column: (df: org.apache.spark.sql.DataFrame, column: String)org.apache.spark.sql.DataFrame
normalize_all_columns: (df: org.apache.spark.sql.DataFrame, excludeCols: Seq[String])org.apache.spark.sql.DataFrame


In [61]:
df = normalize_all_columns(df)

df: org.apache.spark.sql.DataFrame = [user_id: string, avg_bag_value: double ... 16 more fields]


In [66]:
df.printSchema

root
 |-- user_id: string (nullable = true)
 |-- avg_bag_value: double (nullable = true)
 |-- avg_cart_to_checkout_hours: double (nullable = true)
 |-- avg_cart_to_checkout_days: double (nullable = true)
 |-- avg_order_interval_secs: double (nullable = true)
 |-- avg_order_interval_days: double (nullable = true)
 |-- avg_discount: double (nullable = true)
 |-- lifetime_order_value: double (nullable = true)
 |-- active_weeks: double (nullable = true)
 |-- total_orders: double (nullable = true)
 |-- avg_orders_per_week: double (nullable = true)
 |-- num_l1codes_purchased: double (nullable = true)
 |-- num_l2codes_purchased: double (nullable = true)
 |-- num_bricks_purchased: double (nullable = true)
 |-- total_no_of_returns: double (nullable = true)
 |-- total_value_of_return: double (nullable = true)
 |-- brandname: string (nullable = true)
 |-- score: double (nullable = true)



In [62]:
df.show()

+--------------+--------------------+--------------------------+-------------------------+-----------------------+-----------------------+-----------------+--------------------+------------+------------+--------------------+---------------------+---------------------+--------------------+--------------------+---------------------+--------------------+-----+
|       user_id|       avg_bag_value|avg_cart_to_checkout_hours|avg_cart_to_checkout_days|avg_order_interval_secs|avg_order_interval_days|     avg_discount|lifetime_order_value|active_weeks|total_orders| avg_orders_per_week|num_l1codes_purchased|num_l2codes_purchased|num_bricks_purchased| total_no_of_returns|total_value_of_return|           brandname|score|
+--------------+--------------------+--------------------------+-------------------------+-----------------------+-----------------------+-----------------+--------------------+------------+------------+--------------------+---------------------+---------------------+------------

In [65]:
import org.apache.spark.ml.linalg.{Vector, Vectors}
import org.apache.spark.sql.{DataFrame, SparkSession}
import org.apache.spark.sql.functions._

val cosineSim = udf { (v1: Vector, v2: Vector) =>
  val dot = v1.toArray.zip(v2.toArray).map { case (a, b) => a * b }.sum
  val norm1 = math.sqrt(v1.toArray.map(x => x * x).sum)
  val norm2 = math.sqrt(v2.toArray.map(x => x * x).sum)
  if (norm1 == 0.0 || norm2 == 0.0) 0.0 else dot / (norm1 * norm2)
}

import org.apache.spark.ml.linalg.{Vector, Vectors}
import org.apache.spark.sql.{DataFrame, SparkSession}
import org.apache.spark.sql.functions._
cosineSim: org.apache.spark.sql.expressions.UserDefinedFunction = SparkUserDefinedFunction($Lambda$7303/2143819172@2934ff49,DoubleType,List(Some(class[value[0]: vector]), Some(class[value[0]: vector])),Some(class[value[0]: double]),None,false,true)


In [73]:
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.sql.functions.typedLit

def getUserBrandScore(
    spark: SparkSession,
    df: DataFrame,
    userId: String,
    brandName: String,
    topN: Int = 5
): Double = {

  import spark.implicits._

  // List of feature columns (all double columns except user_id, brandname, score)
  val featureCols = df.schema.fields
    .filter(f => f.dataType.simpleString == "double" && !Seq("score").contains(f.name))
    .map(_.name)
    .filterNot(name => name == "brandname" || name == "user_id")

  // Assemble feature vector for each user (ignoring brandname and score)
  val assembler = new VectorAssembler()
    .setInputCols(featureCols)
    .setOutputCol("features")

  val dfWithFeatures = assembler.transform(df)
    .select("user_id", "brandname", "score", "features")
    .cache()

  // 1. Check if score exists for user-brand pair
  val existingScoreOpt = dfWithFeatures
    .filter($"user_id" === userId && $"brandname" === brandName && $"score".isNotNull)
    .select("score")
    .as[Double]
    .collect()
    .headOption

  if (existingScoreOpt.isDefined) {
    existingScoreOpt.get
  } else {
    // 2. Get feature vector of the input user (ignore brandname and score here, just pick first record)
    val userFeatureOpt = dfWithFeatures
      .filter($"user_id" === userId)
      .select("features")
      .collect()
      .headOption
      .map(row => row.getAs[org.apache.spark.ml.linalg.Vector]("features"))

    if (userFeatureOpt.isEmpty) {
      // User features missing → cannot compute similarity → return 0
      0.0
    } else {
      val userFeature = userFeatureOpt.get
        
      val userFeatureBroadcast = spark.sparkContext.broadcast(userFeature)

      // 3. Compute cosine similarity of this user with all others (exclude self)
      val similarities = dfWithFeatures
        .filter($"user_id" =!= userId)
        .withColumn("cosine_sim", cosineSim(typedLit(userFeatureBroadcast.value), $"features"))
        .select("user_id", "brandname", "score", "cosine_sim")

      // 4. Get top N similar users overall (regardless of brand)
      val topSimilarUsers = similarities
        .orderBy(desc("cosine_sim"))
        .limit(topN)
        .select("user_id")
        .as[String]
        .collect()
        .toSet

      if (topSimilarUsers.isEmpty) {
        0.0
      } else {
        // 5. Among these users, get their score for the requested brand
        val scores = dfWithFeatures
          .filter($"brandname" === brandName && $"user_id".isin(topSimilarUsers.toSeq: _*) && $"score".isNotNull)
          .select("score")
          .as[Double]
          .collect()

        if (scores.isEmpty) 0.0 else scores.sum / scores.length
      }
    }
  }
}

import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.sql.functions.typedLit
getUserBrandScore: (spark: org.apache.spark.sql.SparkSession, df: org.apache.spark.sql.DataFrame, userId: String, brandName: String, topN: Int)Double


In [75]:
val predicted = getUserBrandScore(spark, df, "10103222861828", "nike", topN = 10)
println(s"Predicted score for 10103222861828 and nike = $predicted")

Predicted score for 10103222861828 and nike = 1.41


predicted: Double = 1.41
